# zLightGBM

## Inicializacion

In [3]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-12-03 21:37:48 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,720693,38.5,1454584,77.7,1454584,77.7
Vcells,1392144,10.7,8388608,64.0,6349800,48.5


In [4]:
PARAM <- list()
PARAM$experimento <- "final-z-128"
PARAM$semilla_primigenia <- 123457

In [5]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## Preprocesamiento

### Generacion de la clase_ternaria

In [6]:
Sys.time()
require("data.table")

# leo datasets de competencia 2 y 3
dataset_02 <- fread("~/buckets/b1/datasets/competencia_02_crudo.csv.gz")
dataset_03 <- fread("~/buckets/b1/datasets/competencia_03_crudo.csv.gz")

# los uno
dataset <- rbindlist(list(dataset_02, dataset_03), use.names=TRUE, fill=TRUE)

rm(dataset_02, dataset_03)

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  pos = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 + foto_mes%%100 )
]

# ordeno
setorder(dsimple, numero_de_cliente, periodo0)

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0)]
periodo_anteultimo <- periodo_ultimo - 1

# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n = 1:2, fill = NA, type = "lead"),
  by = numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA"]

# calculo BAJA+1
dsimple[
  periodo0 < periodo_ultimo &
    (is.na(periodo1) | periodo0 + 1 < periodo1),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[
  periodo0 < periodo_anteultimo &
    (periodo0 + 1 == periodo1) &
    (is.na(periodo2) | periodo0 + 2 < periodo2),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder(dsimple, pos)
dataset[, clase_ternaria := dsimple$clase_ternaria]

rm(dsimple)
gc()
Sys.time()


[1] "2025-12-03 21:37:48 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,777448,41.6,1454584,77.7,1454584,77.7
Vcells,560599163,4277.1,1481638959,11304.1,1449553697,11059.3


[1] "2025-12-03 21:38:26 UTC"

### Eliminacion de Features

In [7]:
dataset[, mprestamos_personales := NULL ]
dataset[, cprestamos_personales := NULL ]

Sys.time()

[1] "2025-12-03 21:38:26 UTC"

### Feature Engineering

In [8]:
# el mes 1,2, ..12 , podria servir para detectar estacionalidad
dataset[, kmes := foto_mes %% 100]

# creo un ctr_quarter que tenga en cuenta cuando
# los clientes hace 3 menos meses que estan
# ya que seria injusto considerar las transacciones medidas en menor tiempo
dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]
dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5.0]
dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2.0]
dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]

# variable extraida de una tesis de maestria de Irlanda, se perdió el link
dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

In [9]:
# función para imputar NAs en meses con promedio 0
ReemplazarCerosPorNA <- function() {
      # obtener lista de columnas numéricas
    numeric_cols <- names(dataset)[sapply(dataset, is.numeric)]
     # recorrer columnas
  for (col in numeric_cols) {
    # calcular promedio por mes
    promedios <- dataset[, .(prom = mean(get(col), na.rm = TRUE)), by = foto_mes]
# identificar los meses con promedio = 0
    meses_cero <- promedios[prom == 0, foto_mes]

    # reemplazar por NA en esos meses
    if (length(meses_cero) > 0) {
      dataset[foto_mes %in% meses_cero, (col) := NA]
    }
  }
}

In [10]:
ReemplazarCerosPorNA()

## Data Drifting

In [11]:
numeric_cols <- names(dataset)[sapply(dataset, is.numeric)]
numeric_cols

[1] "numero_de_cliente"                   
  [2] "foto_mes"                            
  [3] "active_quarter"                      
  [4] "cliente_vip"                         
  [5] "internet"                            
  [6] "cliente_edad"                        
  [7] "cliente_antiguedad"                  
  [8] "mrentabilidad"                       
  [9] "mrentabilidad_annual"                
 [10] "mcomisiones"                         
 [11] "mactivos_margen"                     
 [12] "mpasivos_margen"                     
 [13] "cproductos"                          
 [14] "tcuentas"                            
 [15] "ccuenta_corriente"                   
 [16] "mcuenta_corriente_adicional"         
 [17] "mcuenta_corriente"                   
 [18] "ccaja_ahorro"                        
 [19] "mcaja_ahorro"                        
 [20] "mcaja_ahorro_adicional"              
 [21] "mcaja_ahorro_dolares"                
 [22] "cdescubierto_preacordado"            
 [23] "mcuentas_saldo"                      
 [24] "ctarjeta_debito"                     
 [25] "ctarjeta_debito_transacciones"       
 [26] "mautoservicio"                       
 [27] "ctarjeta_visa"                       
 [28] "ctarjeta_visa_transacciones"         
 [29] "mtarjeta_visa_consumo"               
 [30] "ctarjeta_master"                     
 [31] "ctarjeta_master_transacciones"       
 [32] "mtarjeta_master_consumo"             
 [33] "cprestamos_prendarios"               
 [34] "mprestamos_prendarios"               
 [35] "cprestamos_hipotecarios"             
 [36] "mprestamos_hipotecarios"             
 [37] "cplazo_fijo"                         
 [38] "mplazo_fijo_dolares"                 
 [39] "mplazo_fijo_pesos"                   
 [40] "cinversion1"                         
 [41] "minversion1_pesos"                   
 [42] "minversion1_dolares"                 
 [43] "cinversion2"                         
 [44] "minversion2"                         
 [45] "cseguro_vida"                        
 [46] "cseguro_auto"                        
 [47] "cseguro_vivienda"                    
 [48] "cseguro_accidentes_personales"       
 [49] "ccaja_seguridad"                     
 [50] "cpayroll_trx"                        
 [51] "mpayroll"                            
 [52] "mpayroll2"                           
 [53] "cpayroll2_trx"                       
 [54] "ccuenta_debitos_automaticos"         
 [55] "mcuenta_debitos_automaticos"         
 [56] "ctarjeta_visa_debitos_automaticos"   
 [57] "mttarjeta_visa_debitos_automaticos"  
 [58] "ctarjeta_master_debitos_automaticos" 
 [59] "mttarjeta_master_debitos_automaticos"
 [60] "cpagodeservicios"                    
 [61] "mpagodeservicios"                    
 [62] "cpagomiscuentas"                     
 [63] "mpagomiscuentas"                     
 [64] "ccajeros_propios_descuentos"         
 [65] "mcajeros_propios_descuentos"         
 [66] "ctarjeta_visa_descuentos"            
 [67] "mtarjeta_visa_descuentos"            
 [68] "ctarjeta_master_descuentos"          
 [69] "mtarjeta_master_descuentos"          
 [70] "ccomisiones_mantenimiento"           
 [71] "mcomisiones_mantenimiento"           
 [72] "ccomisiones_otras"                   
 [73] "mcomisiones_otras"                   
 [74] "cforex"                              
 [75] "cforex_buy"                          
 [76] "mforex_buy"                          
 [77] "cforex_sell"                         
 [78] "mforex_sell"                         
 [79] "ctransferencias_recibidas"           
 [80] "mtransferencias_recibidas"           
 [81] "ctransferencias_emitidas"            
 [82] "mtransferencias_emitidas"            
 [83] "cextraccion_autoservicio"            
 [84] "mextraccion_autoservicio"            
 [85] "ccheques_depositados"                
 [86] "mcheques_depositados"                
 [87] "ccheques_emitidos"                   
 [88] "mcheques_emitidos"                   
 [89] "ccheques_depositados_rechazados"   

In [12]:
drift_rank_cero_fijo <- function(campos_drift) {

  cat( "inicio drift_rank_cero_fijo()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[get(campo) == 0, paste0(campo, "_rank") := 0]
    dataset[get(campo) > 0, paste0(campo, "_rank") :=
      frank(get(campo), ties.method = "random") / .N, by = list(foto_mes)]

    dataset[get(campo) < 0, paste0(campo, "_rank") :=
      -frank(-get(campo), ties.method = "random") / .N, by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat("\n")
  cat( "fin drift_rank_cero_fijo()\n")
}


In [13]:
campos_monetarios <- colnames(dataset)
campos_monetarios <- campos_monetarios[campos_monetarios %like%
  "^(m|Visa_m|Master_m|vm_m)"]

campos_monetarios

[1] "mrentabilidad"                       
 [2] "mrentabilidad_annual"                
 [3] "mcomisiones"                         
 [4] "mactivos_margen"                     
 [5] "mpasivos_margen"                     
 [6] "mcuenta_corriente_adicional"         
 [7] "mcuenta_corriente"                   
 [8] "mcaja_ahorro"                        
 [9] "mcaja_ahorro_adicional"              
[10] "mcaja_ahorro_dolares"                
[11] "mcuentas_saldo"                      
[12] "mautoservicio"                       
[13] "mtarjeta_visa_consumo"               
[14] "mtarjeta_master_consumo"             
[15] "mprestamos_prendarios"               
[16] "mprestamos_hipotecarios"             
[17] "mplazo_fijo_dolares"                 
[18] "mplazo_fijo_pesos"                   
[19] "minversion1_pesos"                   
[20] "minversion1_dolares"                 
[21] "minversion2"                         
[22] "mpayroll"                            
[23] "mpayroll2"                           
[24] "mcuenta_debitos_automaticos"         
[25] "mttarjeta_visa_debitos_automaticos"  
[26] "mttarjeta_master_debitos_automaticos"
[27] "mpagodeservicios"                    
[28] "mpagomiscuentas"                     
[29] "mcajeros_propios_descuentos"         
[30] "mtarjeta_visa_descuentos"            
[31] "mtarjeta_master_descuentos"          
[32] "mcomisiones_mantenimiento"           
[33] "mcomisiones_otras"                   
[34] "mforex_buy"                          
[35] "mforex_sell"                         
[36] "mtransferencias_recibidas"           
[37] "mtransferencias_emitidas"            
[38] "mextraccion_autoservicio"            
[39] "mcheques_depositados"                
[40] "mcheques_emitidos"                   
[41] "mcheques_depositados_rechazados"     
[42] "mcheques_emitidos_rechazados"        
[43] "matm"                                
[44] "matm_other"                          
[45] "Master_mfinanciacion_limite"         
[46] "Master_msaldototal"                  
[47] "Master_msaldopesos"                  
[48] "Master_msaldodolares"                
[49] "Master_mconsumospesos"               
[50] "Master_mconsumosdolares"             
[51] "Master_mlimitecompra"                
[52] "Master_madelantopesos"               
[53] "Master_madelantodolares"             
[54] "Master_mpagado"                      
[55] "Master_mpagospesos"                  
[56] "Master_mpagosdolares"                
[57] "Master_mconsumototal"                
[58] "Master_mpagominimo"                  
[59] "Visa_mfinanciacion_limite"           
[60] "Visa_msaldototal"                    
[61] "Visa_msaldopesos"                    
[62] "Visa_msaldodolares"                  
[63] "Visa_mconsumospesos"                 
[64] "Visa_mconsumosdolares"               
[65] "Visa_mlimitecompra"                  
[66] "Visa_madelantopesos"                 
[67] "Visa_madelantodolares"               
[68] "Visa_mpagado"                        
[69] "Visa_mpagospesos"                    
[70] "Visa_mpagosdolares"                  
[71] "Visa_mconsumototal"                  
[72] "Visa_mpagominimo"                    
[73] "mpayroll_sobre_edad"

In [14]:
drift_rank_cero_fijo(campos_monetarios)

inicio drift_rank_cero_fijo()
mrentabilidad  mrentabilidad_annual  mcomisiones  mactivos_margen  mpasivos_margen  mcuenta_corriente_adicional  mcuenta_corriente  mcaja_ahorro  mcaja_ahorro_adicional  mcaja_ahorro_dolares  mcuentas_saldo  mautoservicio  mtarjeta_visa_consumo  mtarjeta_master_consumo  mprestamos_prendarios  mprestamos_hipotecarios  mplazo_fijo_dolares  mplazo_fijo_pesos  minversion1_pesos  minversion1_dolares  minversion2  mpayroll  mpayroll2  mcuenta_debitos_automaticos  mttarjeta_visa_debitos_automaticos  mttarjeta_master_debitos_automaticos  mpagodeservicios  mpagomiscuentas  mcajeros_propios_descuentos  mtarjeta_visa_descuentos  mtarjeta_master_descuentos  mcomisiones_mantenimiento  mcomisiones_otras  mforex_buy  mforex_sell  mtransferencias_recibidas  mtransferencias_emitidas  mextraccion_autoservicio  mcheques_depositados  mcheques_emitidos  mcheques_depositados_rechazados  mcheques_emitidos_rechazados  matm  matm_other  Master_mfinanciacion_limite  Master_msaldoto

In [15]:
# Feature Engineering Historico
# Creacion de LAGs y Delta LAGs 1,2,3,6,12

setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- setdiff(
 colnames(dataset),
 c("numero_de_cliente","foto_mes","clase_ternaria")
)

# ordenes de lag que quiero
ordenes_lag <- c(1,2,3,6,12)

# genero los lags
for(k in ordenes_lag){
 dataset[,
  paste0(cols_lagueables,"_lag",k) := shift(.SD, k, NA, "lag"),
  by = numero_de_cliente,
  .SDcols = cols_lagueables
 ]
}

# genero los delta lags
for(vcol in cols_lagueables){
 for(k in ordenes_lag){
  dataset[, paste0(vcol,"_delta",k) := get(vcol) - get(paste0(vcol,"_lag",k))]
 }
}

Sys.time()


[1] "2025-12-03 21:46:37 UTC"

In [16]:
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

Loading required package: Rcpp



In [17]:
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      {
        xsum  += x[h] ;
        ysum  += y[h] ;
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")

In [18]:
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = FALSE,
    minimo = FALSE, maximo = FALSE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}

In [19]:
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 6
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- FALSE
PARAM$FE_hist$Tendencias$maximo <- FALSE
PARAM$FE_hist$Tendencias$promedio <- TRUE
PARAM$FE_hist$Tendencias$ratioavg <- FALSE
PARAM$FE_hist$Tendencias$ratiomax <- FALSE

In [20]:
# aqui se agregan las tendencias de los ultimos 6 meses

cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

[1] 1992

[1] "2025-12-03 21:48:18 UTC"

In [21]:
vars <- c(
"ctrx_quarter_normalizado",
"mcaja_ahorro_rank",
"ctrx_quarter",
"mtarjeta_visa_consumo_rank",
"mcuenta_corriente_rank",
"Visa_status",
"cpayroll_trx",
"foto_mes",
"mcuentas_saldo_rank",
"mpayroll_rank",
"mactivos_margen_rank",
"cdescubierto_preacordado",
"Visa_Finiciomora",
"Visa_status_tend6",
"ctarjeta_visa_transacciones",
"cdescubierto_preacordado_tend6",
"mpasivos_margen_rank",
"Visa_mpagominimo_rank",
"ccomisiones_mantenimiento_tend6"
)


In [22]:
#RATIOS ENTRE LAS PRIMERAS 20 VARIABLES DEL FEATURE IMPORTANCE
for(a in vars){
 for(b in vars){
  if(a != b){
   nuevo_nombre <- paste0(a, "_sobre_", b)
   dataset[, (nuevo_nombre) := get(a) / fifelse(get(b)==0, NA_real_, get(b))]
  }
 }
}

In [23]:
#MULTIPLICACIONES
for(a in vars){
 for(b in vars){
  if(a != b){
   nuevo_nombre <- paste0(a, "_por_", b)
   dataset[, (nuevo_nombre) := get(a) * get(b)]
  }
 }
}

# Modelado

## Produccion

In [24]:
# modo de ejecución: "backtest_202107" o "produccion"
#MODO <- "backtest_202107"
MODO <- "produccion"  

In [25]:
Sys.time()

if(MODO == "backtest_202107"){

  # ENTRENAR HASTA 202105 Y PROBAR EN 202107
  PARAM$train_final$meses <- c(
    201901, 201902, 201903, 201904, 201905, 201906,
    201907, 201908, 201909, 201910, 201911, 201912,
    202001, 202002, 202003, 202004, 202005, 202006,
    202007, 202008, 202009, 202010, 202011, 202012,
    202101, 202102, 202103, 202104, 202105
  )

  PARAM$future <- c(202107)  

} else if(MODO == "produccion"){

  # VERSIÓN DE PRODUCCIÓN (LA QUE MOSTRASTE EN LA CAPTURA)
  PARAM$train_final$meses <- c(
    201901, 201902, 201903, 201904, 201905, 201906,
    201907, 201908, 201909, 201910, 201911, 201912,
    202001, 202002, 202003, 202004, 202005, 202006,
    202007, 202008, 202009, 202010, 202011, 202012,
    202101, 202102, 202103, 202104, 202105, 202106,
    202107
  )

  PARAM$future <- c(202109)
}

PARAM$train_final$undersampling <- 0.05

Sys.time()

[1] "2025-12-03 21:49:46 UTC"

[1] "2025-12-03 21:49:46 UTC"

### Final Training Strategy

In [26]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$meses]

In [27]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [28]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Final Model

In [29]:
# utilizo  zLightGBM 
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")
Sys.time()

Loading required package: zlightgbm



[1] "2025-12-03 21:50:07 UTC"

Un hiperparámetro de  *zLightGBM* es la cantidad de canaritos, en este caso se decidió (con tests realizados en {202101, 202102} fijarlo en 100
<br> esta es una importante decisión que debe tomarse
<br> en la version actual de *zLightGBM* los canaritos deben agregarse por fuera al dataset, y mandatoriamente deben ser las primeras columnas del mismo
<br> durante noviembre, si hay suerte, se liberará una versiónn que lo haga automáticamente internamente en el código C++ de la librería

In [30]:
# canaritos
PARAM$qcanaritos <- 5

cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

for( i in seq(PARAM$qcanaritos) ){
  dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
}

# las columnas canaritos mandatoriamente van al comienzo del dataset
cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )

Sys.time()

[1] "2025-12-03 21:50:08 UTC"

In [31]:
# los campos que se van a utilizar

campos_buenos <- setdiff(
  colnames(dataset_train_final),
  c("clase_ternaria", "clase01", "training")
)

In [32]:
# dejo los datos en el formato que necesita LightGBM

dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 246233 columnas 2680 


[1] "2025-12-03 21:50:20 UTC"

Nuevos hiperparámetros en  *zLightGBM*
* canaritos , entero >=0,  cantidad de atributos del dataset que se consideran como canaritos, mandatoriamente en la version actual deben estar al comienzo.
* gradient_bound , numero real, >=0  cota que se pone a los scores de las hojas de los arboles, es un learning_rate adaptativo

En el caso que  canaritos==0  y  gradient_bound==0  entonces  xLightGBM se comporta exactamente igual a  LightGBM

Tener en cuenta que el hiperparámetro  min_data_in_leaf se está dejando en el valor de 20 que es el default de LightGBM
<br> realmente valdria la pena experimentar con distintos valores de  min_data_in_leaf, hay potencial de mejora !

In [33]:
# definicion de parametros, los viejos y los nuevos

PARAM$lgbm <-  list(
  boosting= "gbdt",
  objective= "binary",
  metric= "custom",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE,
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_bin= 31L,
  min_data_in_leaf= 500,  # o 500 para hacer más rápido

  num_iterations= 128, # # pocos arbolitos: el semillerio funciona bien con modelos chiquitos
  num_leaves= 999L, # dejo libre la cantidad de hojas, zLightGBM sabe cuando no hacer un split
  learning_rate= 6,  # se lo deja en 1.0 para que si el score esta por debajo de gradient_bound no se lo escale // 4 experimento fuera de lo común.

  feature_fraction= 0.20, 

  qcanaritos= PARAM$qcanaritos, # fundamental en zLightGBM, aqui esta el control del overfitting
  gradient_bound= 0.2, # 0,1 porque forma parte de un semillerio, 150 prueba fuera de lo común
  canaritos_voracidad = 5  
)

Sys.time()

[1] "2025-12-03 21:50:20 UTC"

In [34]:
# Semillerio Final
PARAM$train_final$ksemillerio  <- 50

PARAM$train_final$cortes <- c(10000,10500, 11000, 11500, 12000,12500)

In [35]:
if(!require("primes")) install.packages("primes")
require("primes")

Loading required package: primes



In [36]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
# me quedo con PARAM$semillerio  primos al azar
PARAM$BO$semillas <- sample(primos)[seq( PARAM$hipeparametertuning$ksemillerio * PARAM$hipeparametertuning$repe )]

cat( PARAM$BO$semillas)

In [37]:
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")

# genero exactamente ksemillerio semillas para el SEMILLERIO
PARAM$train_final$semillas <- sample(primos)[seq(PARAM$train_final$ksemillerio)]

PARAM$train_final$semillas


[1] 597889 599023 234341 941989 443189 793979 652241 324689 509359 225637
[11] 620401 773209 871987 167191 574489 249017 549247 163211 914789 544627
[21] 272761 693437 860789 834949 158359 234977 451873 539641 663961 704453
[31] 381103 190523 121921 849271 967511 269791 171947 210317 728027 303073
[41] 832721 197479 144013 943729 851041 516127 155707 202799 631153 609379

In [38]:
# dejo los datos en formato LightGBM
dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 246233 columnas 2680 


[1] "2025-12-03 21:50:32 UTC"

####  Entrenamiento del modelo

Aqui se ejecuta  *zLightGBM*
<br> no se utilizan hiperparámetros optimos para controlar el overfitting
<br> zLightGBM  sabe cuando no hacer un split
<br> si al hacer el n-simo arbol, no puede hacer el split de la raiz, detiene el crecimiento del ensemble y termina
<br>
<br> claramente el  min_data_in_leaf=20 que es el default de LightGBM esta jugando un papel importante

In [39]:
library(data.table)

dir.create("modelitos", showWarnings = FALSE)

# tabla donde voy a guardar el resumen del semillerio
tb_resumen_semillerio <- data.table(
  semilla = integer(),
  n_arbolitos = integer(),
  hojas_min = integer(),
  hojas_max = integer(),
  hojas_prom = numeric()
)

Sys.time()

for(sem in PARAM$train_final$semillas){

  PARAM$lgbm$seed <- sem
  set.seed(sem, kind = "L'Ecuyer-CMRG")

  cat("\nEntrenando modelo del semillerio con semilla =", sem, "\n")

  modelo_final <- lgb.train(
    data = dtrain_final,
    param = PARAM$lgbm
  )

  # guardo el modelo de ESTA semilla
  arch_modelo <- sprintf("./modelitos/mod_%s.txt", sem)
  lgb.save(modelo_final, file = arch_modelo)

  # ============================
  # detalle de arbolitos por semilla
  # ============================

  tb_arboles <- lgb.model.dt.tree(modelo_final)

  n_arbolitos <- tb_arboles[, max(tree_index) + 1]

  hojas_por_arbol <- tb_arboles[
    ,
    .(hojas = max(leaf_index, na.rm = TRUE) + 1),
    by = tree_index
  ]

  hojas_min  <- min(hojas_por_arbol$hojas)
  hojas_max  <- max(hojas_por_arbol$hojas)
  hojas_prom <- mean(hojas_por_arbol$hojas)

  # muestro en pantalla el dato para CADA semilla
  cat(
    "semilla", sem,
    "| arbolitos =", n_arbolitos,
    "| hojas_prom =", round(hojas_prom, 2),
    "| hojas_max =", hojas_max, "\n"
  )

  # guardo en la tabla resumen
  tb_resumen_semillerio <- rbind(
    tb_resumen_semillerio,
    data.table(
      semilla = sem,
      n_arbolitos = n_arbolitos,
      hojas_min = hojas_min,
      hojas_max = hojas_max,
      hojas_prom = hojas_prom
    )
  )

  rm(modelo_final, tb_arboles, hojas_por_arbol)
  gc()
}

Sys.time()

# opcional: guardo el resumen del semillerio en un archivo
fwrite(tb_resumen_semillerio,
       file = "resumen_semillerio_arbolitos.txt",
       sep = "\t")

cat("\nOJO: el semillerio funciona bien cuando cada modelo tiene pocos arbolitos.\n")


[1] "2025-12-03 21:50:32 UTC"


Entrenando modelo del semillerio con semilla = 597889 
semilla 597889 | arbolitos = 107 | hojas_prom = 354.12 | hojas_max = 399 

Entrenando modelo del semillerio con semilla = 599023 
semilla 599023 | arbolitos = 112 | hojas_prom = 354.63 | hojas_max = 403 

Entrenando modelo del semillerio con semilla = 234341 
semilla 234341 | arbolitos = 114 | hojas_prom = 356.23 | hojas_max = 410 

Entrenando modelo del semillerio con semilla = 941989 
semilla 941989 | arbolitos = 122 | hojas_prom = 347.53 | hojas_max = 401 

Entrenando modelo del semillerio con semilla = 443189 
semilla 443189 | arbolitos = 114 | hojas_prom = 363.35 | hojas_max = 403 

Entrenando modelo del semillerio con semilla = 793979 
semilla 793979 | arbolitos = 114 | hojas_prom = 364.41 | hojas_max = 402 

Entrenando modelo del semillerio con semilla = 652241 
semilla 652241 | arbolitos = 119 | hojas_prom = 359.41 | hojas_max = 406 

Entrenando modelo del semillerio con semilla = 324689 
semilla 324689 | arbolitos = 114 |

[1] "2025-12-03 22:58:39 UTC"


OJO: el semillerio funciona bien cuando cada modelo tiene pocos arbolitos.


### Scoring

Se hace el predict() del modelo en los datos del futuro

In [40]:
# ==========================================================
# FUTURO + CANARITOS
# ==========================================================

# dfuture: solo los meses que querés predecir
dfuture <- dataset[ foto_mes %in% PARAM$future ]

# penosamente, en la versión actual de zLightGBM los campos canaritos
# aunque no se utilizan para nada, también deben estar en el dataset donde se hace el predict()
filas <- nrow(dfuture)

for( i in seq(PARAM$qcanaritos) ){
  dfuture[, paste0("canarito_", i) := runif(filas) ]
}

# matriz de features en el mismo orden que en el entrenamiento
mfuture <- data.matrix(dfuture[, campos_buenos, with = FALSE])

# ==========================================================
# SEMILLERÍO / ENSEMBLE DE MODELOS
# ==========================================================

# acá voy a acumular las predicciones del SEMILLERIO
vpred_ens <- rep(0.0, nrow(dfuture))
qmodelos  <- 0L

for( sem in PARAM$train_final$semillas ) {
  arch_modelo <- sprintf("./modelitos/mod_%s.txt", sem)
  if( file.exists(arch_modelo) ) {
    modelo_final <- lgb.load(arch_modelo)
    p <- predict(modelo_final, mfuture)

    vpred_ens <- vpred_ens + p
    qmodelos  <- qmodelos + 1L

    rm(modelo_final)
    gc()
  }
}

if( qmodelos == 0L ) stop("No se cargó ningún modelo, revisar semillas / archivos mod_*.txt")

# promedio final del ENSEMBLE del semillerio (promedio de probabilidades)
vpred_ens <- vpred_ens / qmodelos
dfuture[, prob_ens := vpred_ens ]

# ==========================================================
# GUARDAR PROBABILIDADES PARA HIBRIDACIÓN
# ==========================================================

tb_prediccion <- dfuture[, .(numero_de_cliente, foto_mes, prob_ens) ]

fwrite(
  tb_prediccion,
  file = "./predicciones_semillerio_lightgbm.txt",
  sep  = "\t"
)

# ==========================================================
# GENERAR ARCHIVOS SEGÚN CORTES
# ==========================================================

# ordeno por probabilidad descendente UNA sola vez
setorder(dfuture, -prob_ens)

# uso los cortes definidos en PARAM$train_final
cortes <- PARAM$train_final$cortes

dir.create("kaggle", showWarnings = FALSE)

for (envios in cortes) {
  envios_efectivos <- min(envios, nrow(dfuture))

  dfuture[, Predicted := 0L]
  if (envios_efectivos > 0L) {
    dfuture[1:envios_efectivos, Predicted := 1L]
  }

  archivo_kaggle <- sprintf("./kaggle/KA_%s_%d.csv",
                            PARAM$experimento, envios)

  fwrite(
    dfuture[, .(numero_de_cliente, Predicted)],
    file = archivo_kaggle,
    sep  = ","
  )

  cat("Generado:", archivo_kaggle,
      "con", envios_efectivos, "envíos\n")
}

Sys.time()


Generado: ./kaggle/KA_final-z-iter128-backtest202107_10000.csv con 10000 envíos
Generado: ./kaggle/KA_final-z-iter128-backtest202107_10500.csv con 10500 envíos
Generado: ./kaggle/KA_final-z-iter128-backtest202107_11000.csv con 11000 envíos
Generado: ./kaggle/KA_final-z-iter128-backtest202107_11500.csv con 11500 envíos
Generado: ./kaggle/KA_final-z-iter128-backtest202107_12000.csv con 12000 envíos
Generado: ./kaggle/KA_final-z-iter128-backtest202107_12500.csv con 12500 envíos


[1] "2025-12-03 23:02:49 UTC"

# Backtesting

In [41]:
if(MODO == "backtest_202107"){

  # dfuture es 202107, con prob_ens ya calculada por el semillerio
  # y con clase_ternaria conocida (porque es backtest)

  setorder(dfuture, -prob_ens)

  gan_unitaria <- ifelse(dfuture$clase_ternaria == "BAJA+2", 780000, -20000.0)
  gan_acum     <- cumsum(gan_unitaria)

  corte_optimo    <- which.max(gan_acum)
  ganancia_optima <- gan_acum[corte_optimo]

  cat("\n========== BACKTEST 202107 ==========\n")
  cat("Mejor corte (número de envíos)  :", corte_optimo, "\n")
  cat("Ganancia máxima en ese corte    :", ganancia_optima, "\n\n")

  cortes_bt <- PARAM$train_final$cortes
  cortes_bt <- cortes_bt[cortes_bt <= nrow(dfuture)]

  tb_backtest_cortes <- data.table(
    corte    = cortes_bt,
    ganancia = gan_acum[cortes_bt]
  )

  cat("Ganancia en cortes predefinidos (backtest 202107):\n")
  print(tb_backtest_cortes)

}



========== BACKTEST 202107 ==========
Mejor corte (número de envíos)  : 11862 
Ganancia máxima en ese corte    : 437160000 

Ganancia en cortes predefinidos (backtest 202107):
   corte  ganancia
   <num>     <num>
1: 10000 424800000
2: 10500 424400000
3: 11000 430400000
4: 11500 433200000
5: 12000 434400000
6: 12500 434000000
